# orbittools
Orbittools is a set of functions useful in working with 2-body problems and observations.  It's not not comprehensive nor particularly fancy, but it is useful.  Bascially I wanted a place to store and easily call functions I used all the time.  I'll update it sometimes.

In [1]:
from orbittools.orbittools import *

orbittools contains some basic functions that it's nice to automate.  Here I'll show what they all do and what inputs look like. <br><br>

**period** uses Kepler's third law to compute the period of a test particle with a certain semi-major axis in a Keplerian orbit around a central mass.  It can take astropy unit objects of any distance and mass and returns period in years:

In [2]:
period(1*u.au,1*u.Msun)

<Quantity 1. yr>

In [3]:
period(149.60e6*u.km,2e30*u.kg)

<Quantity 0.99713598 yr>

Or if you enter values without units, it will return a number in years without an astropy unit.  You must enter semi-major axis in au and mass in solar masses to get the right answer

In [6]:
# With sma in AU and mass in Msol:
period(1,1)
# returns correct period in years

1.0

In [8]:
# With both in mks units:
period(149.60e6,2e30)
# returns the wrong answer

0.0012938454188967088

---
**distance** uses the Bayesian estimation formulation given in Bailer-Jones 2015 to compute distance + error in parsecs given parallax + error in mas.  Designed to work with the output of Gaia parallaxes. <br><br>
For example the distance to HR 8799 using Gaia's parallax is:

In [9]:
# Parallax and error in mas:
distance(24.217514232723282,0.08809423513976626)

(41.29350566835295, 0.15020740717277492)

------
**to_polar** converts RA/DEC in degrees of two objects into their relative separation in mas and position angle in degrees.

For example, the wide stellar binary DS Tuc A and B both have well-defined solutions in Gaia DR2, and their separation and position angle is:

In [10]:
deg_to_mas = 3600000.
mas_to_deg = 1./3600000.

RAa, RAaerr = 354.9154672903039, 0.03459837195273078*mas_to_deg
DECa, DECaerr = -69.19604296286967, 0.02450688383611924*mas_to_deg
RAb, RAberr = 354.914570528965, 0.028643873627224457*mas_to_deg
DECb, DECberr = -69.19458723113503, 0.01971674184397741*mas_to_deg

to_polar(RAa,RAb,DECa,DECb)

(<Quantity 5364.61187229 mas>, <Quantity 347.65815486 deg>)

You can do a quick Monte Carlo to get errors:

In [11]:
seppa = to_polar(np.random.normal(RAa,RAaerr,10000),
                 np.random.normal(RAb,RAberr,10000),
                 np.random.normal(DECa,DECaerr,10000),
                 np.random.normal(DECb,DECberr,10000))

print('Separation =',np.median(seppa[0]),'+-',np.std(seppa[0]))
print('PA =',np.median(seppa[1]),'+-',np.std(seppa[1]))

Separation = 5364.612194537264 mas +- 0.03106503026279128 mas
PA = 347.6581543294433 deg +- 0.00018115002210299626 deg


------
**physical_separation** takes in the distance and angular separation between two objects and returns their physical separation in au.  Distance and angle must be astropy units.

In [12]:
physical_separation(4.5*u.lyr,230*u.mas)

<Quantity 0.31733244 AU>

**angular_separation** takes in distance and physical separation and returns angular separation in arcsec.  Distance and separation must be in astropy units.

In [13]:
angular_separation(4.5*u.lyr,0.317*u.au)

<Quantity 0.22975905 arcsec>

-------
**keplerian_to_cartesian** takes in keplerian orbital elements and returns the observed 3D position, velocity, and acceleration vectors in a right-handed system with +X = +DEC, +Y = +RA, +Z = towards the observer.

Let's look at the inputs:

In [14]:
help(keplerian_to_cartesian)

Help on function keplerian_to_cartesian in module orbittools.orbittools:

keplerian_to_cartesian(sma, ecc, inc, argp, lon, meananom, kep)
    Given a set of Keplerian orbital elements, returns the observable 3-dimensional position, velocity, 
    and acceleration at the specified time.  Accepts and arbitrary number of input orbits.  Semi-major 
    axis must be an astropy unit object in physical distance (ex: au, but not arcsec).  The observation
    time must be converted into mean anomaly before passing into function.
    Inputs:
        sma (1xN arr flt) [au]: semi-major axis in au, must be an astropy units object
        ecc (1xN arr flt) [unitless]: eccentricity
        inc (1xN arr flt) [deg]: inclination
        argp (1xN arr flt) [deg]: argument of periastron
        lon (1xN arr flt) [deg]: longitude of ascending node
        meananom (1xN arr flt) [radians]: mean anomaly 
        kep (1xN arr flt): kepler constant = mu/m where mu = G*m1*m2 and m = [1/m1 + 1/m2]^-1 . 
        

In [20]:
# orbital elements:
sma = 5.2*u.au
ecc = 0.2
inc = 46
argp = 329
lon = 245
# Pick a reference date:
to = 2017.5*u.yr
# Observation date:
t = 2019.34*u.yr
# Masses of the two objects:
m1 = 1*u.Msun
m2 = 0.2*u.Msun
# Compute Kepler's constant:
mu = c.G*m1*m2
m = (1/m2+1/m1)**(-1)
kep = mu/m
# Compute period using orbittools.period function:
per = period(sma,(m1+m2))
#print(per)

meanmotion = np.sqrt(kep/(sma**3)).to(1/u.s)
meananom = meanmotion*((t-to).to(u.s))

pos, vel, acc = keplerian_to_cartesian(sma,ecc,inc,argp,lon,meananom.value,kep)
print('pos',pos)
print('vel',vel)
print('acc',acc)

pos [ 1.17761827 -3.83254452  2.78245744] AU
vel [11.43785945  5.77753521  8.2060902 ] km / s
acc [-2.27896998  7.3993635  -5.37704747] km / (s yr)


It can also return observables for an array of orbits. <br><br>

Or we can use the **keplersconstant** function to compute Kepler's constant:

In [18]:
# Calc from above:
print(kep.to(u.m**3/u.s**2))
# keplersconstant function:
print(keplersconstant(m1,m2))

1.59254928e+20 m3 / s2
1.59254928e+20 m3 / s2


Let's generate 10 trial orbits using the **draw_orbits** function, which draws an array of orbital parameters from priors described in Pearce et al. 2019.  SMA and Long of Nodes are fixed at 100. AU and 0 deg respectively as part of the Orbits for the Imaptient procedure (OFTI; Blunt et al. 2017), because draw_orbits was written as part of that procedure.  For more, see those papers and the **lofti** python package.

**keplerian_to_cartesian** returns a 3xN array of observables for each of the N orbits input.

In [19]:
m1 = 1*u.Msun
m2 = 0.2*u.Msun
kep = keplersconstant(m1,m2)
obsdate = 2019.34

sma, ecc, inc, argp, lon, orbit_fraction = draw_orbits(10)
meananom = orbit_fraction*2*np.pi

pos, vel, acc = keplerian_to_cartesian(sma*u.au,ecc,inc,argp,lon,meananom,kep)
print('pos',pos)
print('vel',vel)
print('acc',acc)

pos [[  98.24653551   -3.93080093   -5.12152856]
 [  62.65059471 -154.03210979   80.53670923]
 [ -99.98510541    8.65962581  -23.6225345 ]
 [  50.0369777     0.66542004    2.56628366]
 [  -1.33288322  -39.4718893    58.29518851]
 [ -90.56341172   42.05329512  -37.61865038]
 [ -35.67982944  -14.6414141    84.71385244]
 [  77.57152507  -20.54985557   14.91263429]
 [ -40.68857851   71.19262366   24.66974907]
 [ -81.71714264   39.11767133   67.43972405]] AU
vel [[ 1.85487755  1.67165991  2.17804312]
 [-0.60110889 -0.63695121  0.33303416]
 [-2.14184417  0.80110621 -2.18533219]
 [ 5.54227627  0.26665664  1.02839791]
 [-4.38596473 -0.33393876  0.49318701]
 [ 1.88580951  1.78749052 -1.59899434]
 [-3.26853367 -0.21169983  1.22487539]
 [-1.98777794 -2.74325161  1.99072485]
 [-3.77060376  0.02925324  0.01013687]
 [-2.32567312 -0.83912169 -1.44666421]] km / s
acc [[-0.02311544  0.00092475  0.00120488]
 [-0.00223064  0.00548424 -0.00286747]
 [ 0.0204868  -0.00177437  0.0048403 ]
 [-0.08931351 -0.00

----
**cartesian_to_keplerian** takes in the 3D position and velocity array and returns the orbital parameters (as astropy unit objects) that would generate those observables.  As of now, it can only handle a single orbit at a time.

<br><br>
Let's take that single orbit from before:

In [21]:
sma = 5.2*u.au
ecc = 0.2
inc = 46
argp = 329
lon = 245
to = 2017.5*u.yr
t = 2019.34*u.yr
m1 = 1*u.Msun
m2 = 0.2*u.Msun
kep = keplersconstant(m1,m2)
per = period(sma,(m1+m2))

meanmotion = np.sqrt(kep/(sma**3)).to(1/u.s)
meananom = meanmotion*((t-to).to(u.s))
print('mean anomaly:',meananom)

pos, vel, acc = keplerian_to_cartesian(sma,ecc,inc,argp,lon,meananom.value,kep)
print('pos',pos)
print('vel',vel)
print('acc',acc)

mean anomaly: 1.068009452846042
pos [ 1.17761827 -3.83254452  2.78245744] AU
vel [11.43785945  5.77753521  8.2060902 ] km / s
acc [-2.27896998  7.3993635  -5.37704747] km / (s yr)


And compute the orbital elements:

In [22]:
cartesian_to_keplerian(pos,vel,kep)

(<Quantity 5.2 AU>,
 <Quantity 0.2>,
 <Quantity 46. deg>,
 <Quantity 329. deg>,
 <Quantity 245. deg>,
 <Quantity 1.06800951>)

Looks good!

---------------
**kepler_advancer** solves the Kepler initial value problem.  Given an initial position and
       velocity vector (in 2 or 3 dimensions; in any reference frame
        [plane of the sky or plane of the orbit]) at an initial time to,
        compute the position and velocity vector at a later time t in 
        that same frame.  <br>
This allows you to compute the position/velocity at some later time without knowing the orbital elements in advance (unlike **kepler_to_cartesian** that requires all orbital elements to be specified).
        
Let's take the same orbital elements from above.

In [23]:
sma = 5.2*u.au
ecc = 0.2
inc = 46
argp = 329
lon = 245
# Pick some times
# Reference epoch:
to = 2017.5*u.yr
# observation epoch:
t = 2019.34*u.yr
# Pick some masses
m1 = 1*u.Msun
m2 = 0.2*u.Msun
kep = keplersconstant(m1,m2)

Let's compute the position/velociy/acceleration at the reference time to:

In [25]:
# Compute mean anomaly for time = t:
meanmotion = np.sqrt(kep/(sma**3)).to(1/u.s)
meananom = meanmotion*((t-to).to(u.s))
pos1, vel1, acc1 = keplerian_to_cartesian(sma,ecc,inc,argp,lon,meananom.value,kep)
print('At time=t, the pos, vel, and accel are:')
print('pos1',pos1)
print('vel1',vel1)
print('acc1',acc1)

At time=t, the pos, vel, and accel are:
pos1 [ 1.17761827 -3.83254452  2.78245744] AU
vel1 [11.43785945  5.77753521  8.2060902 ] km / s
acc1 [-2.27896998  7.3993635  -5.37704747] km / (s yr)


This will make our initial position and velocity vectors.

Now, let's use **kepler_advancer** to find the position and velocity at some later time using only the initial position/velocity:

In [26]:
help(kepler_advancer)

Help on function kepler_advancer in module orbittools.orbittools:

kepler_advancer(ro, vo, t, k, to=0)
    Initial value problem solver.  Given an initial position and
     velocity vector (in 2 or 3 dimensions; in any reference frame
     [plane of the sky, plane of the orbit]) at an initial time to,
     compute the position and velocity vector at a later time t in 
     that same frame.
    
     Written by Logan A. Pearce, 2020
     
     Parameters:
    -----------
    ro : flt, arr
        initial position vector at time = to; astropy unit object
    vo : flt, arr
        initial velocity vector at time = to; astropy unit object
    t : flt
        future time at which to compute new r,v vectors; 
        astropy unit object
    k : flt
        "Kepler's constant", k = G*(m1+m2); astropy unit object
    to : flt
        initial time for initial values.  Default = 0; 
        astropy unit object
    
    Returns:
    --------
    new_r : flt, arr
        new position vector at tim

In [29]:
# Pick a future time:
t2 = t + 3.*u.yr
# Using pos1 and vel1 as inital values, and t as the initial value time,
# compute new pos/vel vectors using kepler_advancer:
pos_ka, vel_ka = kepler_advancer(pos1,vel1,t2,kep,to=t)

# To check out work, let's use kepler_to_cartesian function to compute
# the pos/vel at the new time using the known orbital elements:
meanmotion = np.sqrt(kep/(sma**3)).to(1/u.s)
meananom = meanmotion*((t2-to).to(u.s))
pos_ktc, vel_ktc, acc_ktc = keplerian_to_cartesian(sma,ecc,inc,argp,lon,meananom.value,kep)

print('pos_ka',pos_ka.to(u.AU))
print('pos_ktc',pos_ktc)
print('vel_ka',vel_ka.to(u.km/u.s))
print('vel_ktc',vel_ktc)

pos_ka [4.59667636 2.77695674 3.09873226] AU
pos_ktc [4.59665204 2.77696489 3.09870585] AU
vel_ka [-1.38418293 10.18039854 -5.75435714] km / s
vel_ktc [-1.3842811  10.18039569 -5.75444804] km / s


Looks good!